In [5]:
# Importing the needed packages
import torch 
from torch import nn
import pandas as pd 
import re 
import numpy as np
from typing import Tuple

# Spliting the data into train and test
from sklearn.model_selection import train_test_split

In [6]:
# Reading the data 
d = pd.read_csv('input/Tweets.csv')

# Keeping the needed columns 
d = d[['airline_sentiment', 'text']]

# Leaving only the positive and the negative sentiments 
d = d[d['airline_sentiment'].isin(['positive', 'negative'])]

# Encoding the sentiments that the negative will be 1 and the positive 0
d['airline_sentiment'] = d['airline_sentiment'].apply(lambda x: 0 if x == 'positive' else 1)

In [7]:
def preprocess_text(x: str) -> str:
    """
    Function that preprocess the text before tokenization

    Args:
        x (str): text to preprocess

    Returns:
        str: preprocessed text
    """ 
    # Create whitespaces around punctuation
    x = re.sub(r'([.,!?;:])', r' \1 ', x)

    # Returns the text 
    return x

In [8]:
# Preprocessing the text
d['text'] = d['text'].apply(preprocess_text)

# Spliting to train test 
train, test = train_test_split(d, test_size=0.2, random_state=42)

# Reseting the indexes 
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')

Train shape: (9232, 2)
Test shape: (2309, 2)


In [9]:
def create_word_index(x: str, shift_for_padding: bool = False) -> Tuple[dict, dict]: 
    """
    Function that scans a given text and creates two dictionaries:
    - word2idx: dictionary mapping words to integers
    - idx2word: dictionary mapping integers to words

    Args:
        x (str): text to scan
        shift_for_padding (bool, optional): If True, the function will add 1 to all the indexes.

    Returns:
        Tuple[dict, dict]: word2idx and idx2word dictionaries
    """
    # Spliting the text into words
    words = x.split()

    # Creating the word2idx dictionary 
    word2idx = {}
    for word in words: 
        if word not in word2idx: 
            # The len(word2idx) will always ensure that the 
            # new index is 1 + the length of the dictionary so far
            word2idx[word] = len(word2idx)

    # Adding the <UNK> token to the dictionary; This token will be used 
    # on new texts that were not seen during training.
    # It will have the last index. 
    word2idx['<UNK>'] = len(word2idx)

    if shift_for_padding:
        # Adding 1 to all the indexes; 
        # The 0 index will be reserved for padding
        word2idx = {k: v + 1 for k, v in word2idx.items()}

    # Reversing the above dictionary and creating the idx2word dictionary
    idx2word = {idx: word for word, idx in word2idx.items()}

    # Returns the dictionaries
    return word2idx, idx2word

In [10]:
# Joining all the texts into one string
text = ' '.join(train['text'].values)

# Creating the word2idx and idx2word dictionaries
word2idx, idx2word = create_word_index(text, shift_for_padding=True)

# Printing the size of the vocabulary
print(f'The size of the vocabulary is: {len(word2idx)}')

The size of the vocabulary is: 15572


In [11]:
# For each row in the train and test set, we will create a list of integers
# that will represent the words in the text
train['text_int'] = train['text'].apply(lambda x: [word2idx.get(word, word2idx['<UNK>']) for word in x.split()])
test['text_int'] = test['text'].apply(lambda x: [word2idx.get(word, word2idx['<UNK>']) for word in x.split()])

# Calculating the length of sequences in the train set 
train['seq_len'] = train['text_int'].apply(lambda x: len(x))

# Describing the length of the sequences
train['seq_len'].describe()

count    9232.000000
mean       21.295494
std         7.311373
min         2.000000
25%        16.000000
50%        23.000000
75%        27.000000
max        43.000000
Name: seq_len, dtype: float64

In [12]:
train

,airline_sentiment,text,text_int,seq_len
0,1,@USAirways Another dead end . They only hand...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 6, ...",29
1,1,@USAirways #2066 . Was on plane from PBI to C...,"[1, 26, 5, 27, 28, 29, 30, 31, 32, 33, 34, 35,...",31
2,1,@USAirways waiting for bags now over 25min in ...,"[1, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 49]",12
3,1,"@USAirways Never heard back , but this would ...","[1, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70...",29
4,0,@united Thanks !,"[83, 84, 49]",3
...,...,...,...,...
9227,1,@AmericanAir she should not only get a refund ...,"[190, 293, 267, 210, 7, 223, 41, 797, 64, 982,...",17
9228,1,@SouthwestAir I need updates on my flights 464...,"[239, 19, 243, 881, 28, 74, 533, 15558, 162, 1...",22
9229,0,Lovely ! RT @JetBlue : Our fleet’s on fleek ...,"[14321, 49, 4479, 134, 68, 545, 15560, 28, 547...",15
9230,1,@united Okay thanks if you could please update...,"[83, 5876, 137, 522, 115, 71, 360, 87, 14, 5, ...",25


In [131]:
def pad_sequences(x: list, pad_length: int) -> list:
    """
    Function that pads a given list of integers to a given length

    Args:
        x (list): list of integers to pad
        pad_length (int): length to pad

    Returns:
        list: padded list of integers
    """
    # Getting the length of the list
    len_x = len(x)

    # Checking if the length of the list is less than the pad_length
    if len_x < pad_length: 
        # Padding the list with 0s
        x = x + [0] * (pad_length - len_x)
    else: 
        # Truncating the list to the desired length
        x = x[:pad_length]

    # Returning the padded list
    return x

In [132]:
# Padding the train and test sequences 
train['text_int'] = train['text_int'].apply(lambda x: pad_sequences(x, 30))
test['text_int'] = test['text_int'].apply(lambda x: pad_sequences(x, 30))

In [133]:
train

,airline_sentiment,text,text_int,seq_len
0,1,@USAirways Another dead end . They only hand...,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 6, ...",29
1,1,@USAirways #2066 . Was on plane from PBI to C...,"[1, 26, 5, 27, 28, 29, 30, 31, 32, 33, 34, 35,...",31
2,1,@USAirways waiting for bags now over 25min in ...,"[1, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 49...",12
3,1,"@USAirways Never heard back , but this would ...","[1, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70...",29
4,0,@united Thanks !,"[83, 84, 49, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3
...,...,...,...,...
9227,1,@AmericanAir she should not only get a refund ...,"[190, 293, 267, 210, 7, 223, 41, 797, 64, 982,...",17
9228,1,@SouthwestAir I need updates on my flights 464...,"[239, 19, 243, 881, 28, 74, 533, 15558, 162, 1...",22
9229,0,Lovely ! RT @JetBlue : Our fleet’s on fleek ...,"[14321, 49, 4479, 134, 68, 545, 15560, 28, 547...",15
9230,1,@united Okay thanks if you could please update...,"[83, 5876, 137, 522, 115, 71, 360, 87, 14, 5, ...",25


In [134]:
# Defining the torch model for sentiment classification 
class SentimentClassifier(torch.nn.Module):
    """
    Class that defines the sentiment classifier model
    """
    def __init__(self, vocab_size, embedding_dim):
        super(SentimentClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size + 1, embedding_dim)
        self.rnn = nn.RNN(input_size=embedding_dim, hidden_size=1, batch_first=True)
        self.fc = nn.Linear(1, 1)  # Output with a single neuron for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation

    def forward(self, x):
        x = self.embedding(x)  # Embedding layer
        output, _ = self.rnn(x)  # RNN layer
        # Use the hidden state from the last time step as the representation of the sequence
        x = output[:, -1, :]

        # Fully connected layer with a single neuron
        x = self.fc(x) 
        
        # Converting to probabilities
        x = self.sigmoid(x)

        # Flattening the output
        x = x.squeeze()
        
        return x

# Initiating the model 
model = SentimentClassifier(vocab_size=len(word2idx), embedding_dim=16)

# Initiating the criterion and the optimizer
criterion = nn.BCELoss() # Binary cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [135]:
# Defining the data loader 
from torch.utils.data import Dataset, DataLoader

class TextClassificationDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # The x is named as text_int and the y as airline_sentiment
        x = self.data.iloc[idx]['text_int']
        y = self.data.iloc[idx]['airline_sentiment']
        
        # Converting the x and y to torch tensors
        x = torch.tensor(x)
        y = torch.tensor(y)

        # Converting the y variable to float 
        y = y.float()

        # Returning the x and y
        return x, y
    
# Creating the train and test loaders
train_loader = DataLoader(TextClassificationDataset(train), batch_size=32, shuffle=True)
test_loader = DataLoader(TextClassificationDataset(test), batch_size=32, shuffle=True)

In [136]:
# Defining the number of epochs
epochs = 10

# Setting the model to train mode
model.train()

# Iterating through epochs
for epoch in range(epochs):
    # Initiating the total loss 
    total_loss = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass

        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update the model's parameters

        # Adding the loss to the total loss
        total_loss += loss.item()

    # Calculating the average loss
    avg_loss = total_loss / len(train_loader)

    # Printing the loss every n epochs
    if epoch % 5 == 0:
        print(f'Epoch: {epoch}, Loss: {avg_loss}')

Epoch: 0, Loss: 0.7827757836213162
Epoch: 5, Loss: 0.5128291124497318
Epoch: 10, Loss: 0.5100423388415142
Epoch: 15, Loss: 0.5085798447313605
Epoch: 20, Loss: 0.5045332652887259
Epoch: 25, Loss: 0.4583601834039787
Epoch: 30, Loss: 0.4495825650988978
Epoch: 35, Loss: 0.4268802881859578


In [137]:
# Setting the model to eval model
model.eval()

# List to track the test acc 
total_correct = 0
total_obs = 0

# Iterating over the test set
for batch_idx, (inputs, labels) in enumerate(test_loader):
    outputs = model(inputs)  # Forward pass

    # Getting the number of correct predictions 
    correct = ((outputs > 0.5).float() == labels).float().sum()

    # Getting the total number of predictions
    total = labels.size(0)

    # Updating the total correct and total observations
    total_correct += correct
    total_obs += total

print(f'The test accuracy is: {total_correct / total_obs}')

The test accuracy is: 0.718492865562439
